In [593]:
import requests
import time

headers = {"User-Agent": "Topic-Extraction/1.0 (https://github.com/luka1220/Bachelor_Thesis) requests/2.22.0"}

def query_paths_to_entity(item):
    return """
    SELECT ?item ?itemLabel ?pre ?superItem ?superItemLabel
    { 
      wd:"""+ item +"""  (wdt:P279 | wdt:P31 | wdt:P361 )* ?item.
      ?item   ( wdt:P279 | wdt:P31 | wdt:P361) ?superItem.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    """

def select_query_service(item, pre):
    return """
    PREFIX bd: <http://www.bigdata.com/rdf#>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    
    SELECT ?item ?itemLabel ?superItem ?superItemLabel
    { 
      wd:"""+ item +""" wdt:"""+ pre +"""* ?item.
      ?item wdt:"""+ pre +""" ?superItem.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    """
def select_query(item, pre):
    return """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    
    SELECT ?item ?label
    WHERE 
    {
        wd:"""+ item +""" wdt:"""+ pre +""" ?item.
        ?item rdfs:label ?label.
        FILTER (lang(?label) = "en")
    }
    """
def query_dbp_wiki(dbpediaConcept):
    return """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbpedia: <http://dbpedia.org/resource/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    SELECT ?obj
    WHERE 
    { 
        <"""+ dbpediaConcept +"""> (owl:sameAs|^owl:sameAs) ?obj
        FILTER(CONTAINS(str(?obj), \"http://www.wikidata.org\")) 
    }
    """

def map_dbp_wikid(dbpediaConcept):
    url = "http://dbpedia.org/sparql"
    results = requests.get(url, headers=headers, params={'query': query_dbp_wiki(dbpediaConcept), 'format': 'json'})
    if results.status_code is not 200:
        raise Exception(results.status_code, results.test)
    
    results = results.json()["results"]["bindings"]
    if len(results)==0:
        print("No results in dbpedia found for " + dbpediaConcept)
        raise Exception("No results in dbpedia found for " + dbpediaConcept)
    if len(results)>1:
        print("Multiple results in dbpedia found for " + dbpediaConcept)
        raise Exception("Multiple results in dbpedia found for " + dbpediaConcept)
    return results[0]["obj"]["value"]

def wikidata_sparql_request(query, counter=0):
    url = 'https://query.wikidata.org/sparql'
    res = requests.get(url, headers=headers, params={'query': query, 'format': 'json'})
    if res.status_code is 200:
        return res.json()["results"]["bindings"]
    elif counter < 5: 
        print(res.status_code, res.text,"herders: ", res.headers)
        if status_code is 429: time.sleep(15)
        elif status_code is 403: time.sleep(20)
        else: raise Exception("Status code is " + str(res.status_code))
        return wikidata_sparql_request(nodeId, predicate, counter+1)
    else: raise Exception("Status code is " + str(res.status_code))

#def get_all_subclasses(nodeId): return wikidata_sparql_request(nodeId,"P279")
#def get_all_instances_of(nodeId): return wikidata_sparql_request(nodeId, "P31")

def convert_dbp_wikid_ids(dbp_concepts):
    return [(map_dbp_wikid(c).split("/")[-1], c.split("/")[-1]) for c in dbp_concepts]

In [628]:
import pydot
import sys

class Node:
    def __init__(self,node_id, value = "NULL"):
        self.id = node_id
        self.value = value
        self.neighbours = []
        self.status = 0
    def getId(self):
        return self.id
    def add_neighbour(self, nkey):
        self.neighbours += [nkey]
    def get_neighbous(self):
        return self.neighbours
    def get_status(self):
        return self.status
    def set_status(self, s):
        self.status = s

# Directed Knoleage Graph
class K_Graph:
    def __init__(self, nodes):
        self.graph = dict([(k, Node(k,v)) for (k,v) in nodes])
        self.concepts = nodes
        self.is_build = False
    def get_graph(self):
        return self.graph
    def build_edge(self, key1, v1, key2, v2):
        if key1 not in self.graph:
            self.graph[key1] = Node(key1, v1)
        if key2 not in self.graph:
            self.graph[key2] = Node(key2, v2)
        if key2 not in self.graph[key1].neighbours:
            self.graph[key1].add_neighbour(key2)
    def build_graph(self):
        if self.is_build: return "Graph is allready build, len: "+ str(len(self.graph))
        
        for i, (key,v) in enumerate(self.concepts):
            text = "\rPercent: {0}%".format( int(i/len(self.concepts) *100))
            sys.stdout.write(text)
            sys.stdout.flush()
            for s in wikidata_sparql_request(query_paths_to_entity(key)):
                self.build_edge(s["item"]["value"].split("/")[-1], 
                                s["itemLabel"]["value"],
                                s["superItem"]["value"].split("/")[-1],
                                s["superItemLabel"]["value"])
        self.is_build = True
        return len(self.graph)
    def get_graphic_G(self):
        G = pydot.Dot(graph_type="digraph")
        node = pydot.Node("Q35120", label="Entity", style="filled", fillcolor="green")
        G.add_node(node)
        for (k,v) in self.concepts:
            G.add_node(pydot.Node(k, label=v, style="filled", fillcolor="yellow"))
        
        #init Nodes
        for n in self.graph.values():
            if G.get_node(n.id) == []:
                v = pydot.Node(n.id, label=n.value)
                G.add_node(v)
            for neighbour in n.neighbours:
                e = pydot.Edge(n.id, neighbour)
                G.add_edge(e)
        return G
            
            
def build_up_G(n, g_dict, Graph):
    if g_dict[n].status is 0:
        g_dict[n].status = 1
    else: return 0
    for neigh in g_dict[n].neighbours:
        e = pydot.Edge(n, neigh)
        Graph.add_edge(e)
        build_up_G(neigh, g_dict, Graph)
    return 1


In [4]:
import json
import numpy as np
with open('ac1-export-complete.json', 'r') as f:
    ac1_dict = json.load(f)["@graph"]
    ac1_dict = list(filter(lambda i: "concept" in i and type(i["concept"]) is list, ac1_dict))
    content = list(map(lambda idea: idea["content"], ac1_dict));
    concept_per_idea = list(map(lambda idea: [c["linkedConcept"] for c in idea["concept"]], ac1_dict));
    concepts = np.array(list(set([y for x in concept_per_idea for y in x])))
    print(len(concepts))

199


In [401]:
wiki_concepts = convert_dbp_wikid_ids(concepts)

In [586]:
graph = K_Graph(wiki_concepts)
graph.build_graph()


Percent: 99%

1133

In [587]:
G = graph.get_graphic_G()

In [518]:
from IPython.display import Image, display

im = Image(G.create_png())
display(im)

In [588]:
G.write_png('knowledge_graph_big.png')

True

In [632]:
g2 = K_Graph(wiki_concepts[2:3])
g2.build_graph()

Percent: 0%

528

In [633]:
G2 = g2.get_graphic_G()
print(len(G2.get_edges()), len(G2.get_nodes()))

1185 528


In [634]:
G2.write_png('test.png')
#im2 = Image(G2.create_png())
#display(im2)

True

In [624]:
wiki_concepts[4]

('Q9893', 'I')